In [17]:
import ee
%matplotlib inline
import math
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
from scipy import stats
import scipy

import datetime, calendar
import spei

#ee.Authenticate()
ee.Initialize()

In [18]:
MODEL_INFO = {'UKESM1-0-LL': 'HadAM',
 'NorESM2-MM': 'CCM',
 'NorESM2-LM': 'CCM',
 'MRI-ESM2-0': 'UCLA GCM',
 'MPI-ESM1-2-LR': 'ECMWF',
 'MPI-ESM1-2-HR': 'ECMWF',
 'MIROC6': 'MIROC',
 'MIROC-ES2L': 'MIROC',
 'KIOST-ESM': 'GFDL',
 'KACE-1-0-G': 'HadAM',
 'IPSL-CM6A-LR': 'IPSL',
 'INM-CM5-0': 'INM',
 'INM-CM4-8': 'INM',
 'HadGEM3-GC31-MM': 'HadAM',
 'HadGEM3-GC31-LL': 'HadAM',
 'GFDL-ESM4': 'GFDL',
 'GFDL-CM4_gr2': 'GFDL',
 'GFDL-CM4': 'GFDL',
 'FGOALS-g3': 'CCM',
 'EC-Earth3-Veg-LR': 'ECMWF',
 'EC-Earth3': 'ECMWF',
 'CanESM5': 'CanAM',
 'CNRM-ESM2-1': 'ECMWF',
 'CNRM-CM6-1': 'ECMWF',
 'CMCC-ESM2': 'CCM',
 'CMCC-CM2-SR5': 'CCM',
 #'BCC-CSM2-MR': 'CCM',
 'ACCESS-ESM1-5': 'HadAM',
 'ACCESS-CM2': 'HadAM',
 'TaiESM1': 'CCM',
}

EXCLUDED_MODELS = ['GFDL-CM4_gr2','ERA5']

MODELS = [i for i in MODEL_INFO.keys() if not i in EXCLUDED_MODELS]


CAMPINAS_LATLON = (-22.907104, -47.063240)  # Campinas

HIST_START = 1980
HIST_END = 2014

PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

NUM_BEST_MODELS = 3

In [19]:
VARIABLES = {
    'tasmax': {
        'era_varname': 'maximum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'tasmin': {
        'era_varname': 'minimum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'pr': {
        'era_varname': 'total_precipitation',
        'nex_transform': lambda x: x * 86400,
        'era_transform': lambda x: x * 1000
    },
    'hurs': {
        'era_varname': None,
       'nex_transform': lambda x: x,
        'era_transform': lambda x: x
    }
}

In [34]:
def calendardate_percentiles(nex_varname, q, latlon, sh_hem=False):
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(get_observed_gee(nex_varname, latlon, start_year=year, end_year=year, southern_hem=False))
    if not sh_hem:
        return np.percentile(np.vstack(allyears), q, axis=0)
    else:
        res = np.percentile(np.vstack(allyears), q, axis=0)
        return np.concatenate([res[152:], res[:152]])

def wholeyear_percentile(nex_varname, q, latlon):
    if not nex_varname == 'ari':
        hist_start = PERCENTILE_STARTYEAR
        hist_end = PERCENTILE_ENDYEAR
        allyears = []
        for year in range(hist_start, hist_end):
            allyears.append(get_observed_gee(nex_varname, latlon, start_year=year, end_year=year, southern_hem=False))
        return np.percentile(np.concatenate(allyears).flatten(), q)
    else:
        hist_start = PERCENTILE_STARTYEAR
        hist_end = PERCENTILE_ENDYEAR
        allyears = []
        for year in range(hist_start, hist_end):
            allyears.append(get_observed_gee('pr', latlon, start_year=year, end_year=year, southern_hem=False))
        ari_data = ari(np.concatenate(allyears).flatten())
        return np.percentile(ari_data, 95)

def yearextreme_percentile(nex_varname, q, latlon, wantmax):
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(get_observed_gee(nex_varname, latlon, start_year=year, end_year=year, southern_hem=False))
    return np.percentile(np.array(allyears), q)

def d2j(datestring):
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday

def removeLeapDays(arr, start_year, end_year, southern_hem):
    indices_to_remove = []
    for year in range(start_year, end_year + 1):
        if calendar.isleap(year):
            indices_to_remove.append(((year-start_year) * 365) + [0,183][int(southern_hem)] + len(indices_to_remove) + 59)
    return np.delete(arr, indices_to_remove)

def get_rmsd(d1, d2):
    c1 = seasonal_means(d1)
    c2 = seasonal_means(d2)
    return np.sqrt(np.mean(np.sum((c1 - c2)**2)))

def count_runs(tf_array, min_runsize):
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    count = 0
    for idx in range(starts.size):
        if ends[idx] - starts[idx] >= min_runsize:
            count += 1
    return count

def longest_run(tf_array):
    if np.sum(tf_array) == 0:
        return 0
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    durations = ends - starts
    return max(durations)
    
def quarters(d, start_year, end_year, southern_hem=False):
    #Takes multi-year array and returns data reorganized into quarters
    q2 = []  # 60-151
    q3 = []  # 152-243
    q4 = []  # 244-334
    q1 = []  # 335-59
    if not southern_hem:
        jan1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
            q1.append(tmp)
            q2.append(d[jan1_idx + 60 : jan1_idx + 152])
            q3.append(d[jan1_idx + 152 : jan1_idx + 244])
            q4.append(d[jan1_idx + 244 : jan1_idx + 335])

            jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q2)
        jja_res = np.vstack(q3)
        son_res = np.vstack(q4)
        djf_res = np.vstack(q1)
    else:
        jul1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jul1_idx - 365 : jul1_idx - 365 + 60], d[jul1_idx + 335 : jul1_idx + 365]), axis=0)
            q3.append(tmp)
            q4.append(d[jul1_idx + 60 : jul1_idx + 152])
            q1.append(d[jul1_idx + 152 : jul1_idx + 244])
            q2.append(d[jul1_idx + 244 : jul1_idx + 335])

            jul1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q4)
        jja_res = np.vstack(q1)
        son_res = np.vstack(q2)
        djf_res = np.vstack(q3)
    return mam_res, jja_res, son_res, djf_res
    
def seasonal_means(d):
    q = quarters(d, HIST_START, HIST_END)
    return np.array([np.mean(q[0], axis=1), np.mean(q[1], axis=1), np.mean(q[2], axis=1), np.mean(q[3], axis=1)])

def calibration_function(hist_obs, hist_mod):
# Calibration functions are P-P plots of historical and modeled values

    source = np.sort(hist_obs.flatten())
    target= np.sort(hist_mod.flatten())
   
    if (np.max(source) == 0 and np.min(source) == 0):
        return np.arange(0, target.size) / target.size
    if (np.max(target) == 0 and np.min(target) == 0):
        return np.arange(0, source.size) / source.size
    new_indices = []

    for target_idx, target_value in enumerate(target):
        if target_idx < len(source):
            source_value = source[target_idx]
            if source_value > target[-1]:
                new_indices.append(target.size - 1)
            else:
                new_indices.append(np.argmax(target >= source_value))
    return np.array(new_indices) / source.size

def calibrate_component(uncalibrated_data, calibration_fxn):
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

In [33]:
def ari(yeardata):
    ARI_WEIGHTS = np.array([
        0.013499274414000246,
        0.01837401239683367,
        0.026458577851440485,
        0.041341527892875755,
        0.07349604958733467,
        0.16536611157150302,
        0.6614644462860121
    ])
    def ari_7day(sevendayrain):
        return np.dot(sevendayrain, ARI_WEIGHTS)
    
    res = []
    for start_idx in range(yeardata.size-7):
        res.append(ari_7day(yeardata[start_idx:start_idx+7]))
    return res

def wetbulbtemp(T, RH):
# JA Knox et al. 2017. Two simple and accurate approximations for wet-bulb
# temperature in moist conditions, with forecasting applications. Bull. Am.
# Meteorol. Soc. 98(9): 1897-1906. doi:10.1175/BAMS-D-16-0246.1
    T = T.astype(np.float64)
    rh_percent = RH.astype(np.float64)
    return T * np.arctan(0.151977 * np.sqrt(rh_percent + 8.313659)) + np.arctan(T + rh_percent) - np.arctan(rh_percent - 1.676331) + ((0.00391838 * ((rh_percent)**(3/2))) * np.arctan(0.023101 * rh_percent)) - 4.686035        


In [97]:
class Hazard:
    def get_expectedval(self, latlon, start_year, end_year, datasets, calib_fxns):
        southern_hem = int(latlon[0] < 0)
        fut_mod = {}
        varnames = self.varname.split('+')
        for varname in varnames:
            for model in calib_fxns[varname].keys():
                ds = datasets[varname][model]

                fut_mod[(varname, model)] = ds
        best_models = []
        for idx in range(NUM_BEST_MODELS):
            modelplus = '+'.join([list(calib_fxns[varname].keys())[idx] for varname in varnames])
            best_models.append(modelplus)

        para_res = {}
        numbins = end_year - start_year + 1
        for modelplus in best_models:
            calib_data = []
            for idx, varname in enumerate(varnames):
                model = modelplus.split('+')[idx]
                calib_data.append(np.array(calibrate(fut_mod[(varname, model)], calib_fxns[varname][model])))
            countdist = self.val_dist([cd[[0,152][int(not southern_hem)]:[len(cd),-213][int(not southern_hem)]] for cd in calib_data])
        
            observed_vals = np.array(list(countdist.keys()))
            minval = observed_vals[0] - 1
            maxval = observed_vals[-1] + 1
            D = (maxval - minval) / (numbins - 1)
            for i in range(numbins):
                centerval = minval + (i * D)
                countdist[centerval] = np.sum(observed_vals >= minval + ((i - 0.5) * D) * (observed_vals < minval + ((i + 0.5) * D)))
            alpha = np.array(list(countdist.values())) + 0.5
            res = []
            for i in range(10000):
                dirich_samp = np.random.dirichlet(alpha, 1)
                mult_samp = np.random.multinomial(end_year - start_year + 1, dirich_samp[0], 1)[0]
                res.append(sum([list(countdist.keys())[j] * mult_samp[j] for j in range(len(list(countdist.keys())))]) / (end_year - start_year + 1))
            res = np.array(res)
            para_res[modelplus] = res

        return {modelplus:[np.mean(para_res[modelplus])- (1.96 * np.std(para_res[modelplus])), np.mean(para_res[modelplus]), np.mean(para_res[modelplus]) + (1.96 * np.std(para_res[modelplus]))] for modelplus in best_models}
    
    def get_exceedanceprob(self, latlon, start_year, end_year, datasets, calib_fxns):
        southern_hem = int(latlon[0] < 0)
        fut_mod = {}
        varnames = self.varname.split('+')
        for varname in varnames:
            for model in calib_fxns[varname].keys():
                ds = datasets[varname][model]
                fut_mod[(varname, model)] = ds
        best_models = []
        for idx in range(NUM_BEST_MODELS):
            modelplus = '+'.join([list(calib_fxns[varname].keys())[idx] for varname in varnames])
            best_models.append(modelplus)

        prob_res = {}
        for modelplus in best_models:
            calib_data = []
            for idx, varname in enumerate(varnames):
                model = modelplus.split('+')[idx]
                calib_data.append(np.array(calibrate(fut_mod[(varname, model)], calib_fxns[varname][model])))
            countdist = self.val_dist([cd[[0,152][int(not southern_hem)]:[len(cd),-213][int(not southern_hem)]] for cd in calib_data])
        
            count = sum([countdist[val] * int((val >= self.extremeval and self.exceed_is_gte) or (val <= self.extremeval and not self.exceed_is_gte)) for val in countdist])
            N = end_year - start_year + 1
            if self.probmodel == 'Poisson':
                meanprob = (count + 0.5) / N
                stdprob = np.sqrt((2 * count) + 1) / N
            else: 
                meanprob = ((count + 0.5) * N / (N + 1)) / N
                stdprob = (np.sqrt((N * (count + 0.5) * (N - count + 0.5) * ((2 * N) + 1)) / ((N + 2) * (N + 1) * (N + 1)))) / N
            prob_res[modelplus] = [meanprob - (1.96 * stdprob), meanprob, meanprob + (1.96 * stdprob)]
       
        return prob_res

class ARIDays(Hazard):
    def __init__(self, hazname, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        vals = apply_along_axis(ari, data, axis=1)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class WetbulbDays(Hazard):
    def __init__(self, hazname, wbt_threshold, extremeval):
        self.hazname = hazname
        self.varname = 'tasmax+hurs'
        self.wbt_threshold = wbt_threshold
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
    
    def val_dist(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        vals = np.sum(byyear >= self.wbt_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class DroughtSPIDays(Hazard):
    def __init__(self, hazname, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
    
    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        
        t=pd.date_range(start='1980-01-01', end='{0}-12-31'.format(1980 + (data.size//365) - 1), freq='D')
        t = t[~((t.month == 2) & (t.day == 29))]
        
        droughtdays = spei.spi(pd.Series(data, index=t)).to_numpy()
        byyear = droughtdays.reshape(data.size // 365, 365)
        
        vals = np.sum(byyear <= -2, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class Tempwave_count(Hazard):
    def __init__(self, hazname, varname, min_duration, threshold, tf_gte, extremeval):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.hazname = hazname
        self.varname = varname
        self.tf_gte = tf_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data = datalist[0]
        if self.tf_gte:
            return data >= self.threshold
        else:
            return data <= self.threshold

    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(count_runs, 1, tfarray, self.min_duration)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class Tempwave_maxduration(Hazard):
    def __init__(self, hazname, varname, threshold, tf_gte, extremeval):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.hazname = hazname
        self.varname = varname
        self.tf_gte = tf_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data = datalist[0]
        if self.tf_gte:
            return data >= self.threshold
        else:
            return data <= self.threshold

    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(longest_run, 1, tfarray)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class Tempwave_highlow_count(Hazard):
    def __init__(self, hazname, hightemp, lowtemp, min_duration, extremeval):
        self.hazname = hazname
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return tf_array_tx * tf_array_tn
    
    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(count_runs, 1, tfarray, self.min_duration)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class Tempwave_highlow_duration(Hazard):
    def __init__(self, hazname, hightemp, lowtemp, min_duration, extremeval):
        self.hazname = hazname
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
        
    def tf_array(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return tf_array_tx * tf_array_tn
    
    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(longest_run, 1, tfarray)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class ThresholdDays(Hazard):
    def __init__(self, hazname, varname, var_threshold, want_max, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_max = want_max
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        if self.want_max:
            vals = np.sum(byyear >= self.var_threshold, axis=1)
        else:
            vals = np.sum(byyear <= self.var_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
        
class Dryduration_seasonal_count(Hazard):
    def __init__(self, hazname, startdate, enddate, southern_hem, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.startdate = startdate
        self.enddate = enddate
        self.southern_hem = southern_hem
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data = datalist[0] == 0
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size//365, 365)
        
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array(inseason_onerow * (data.size//365))
        inseason = inseason.reshape(data.size//365, 365)
        return byyear * inseason

    def val_dist(self, datalist): 
        byyear = self.tf_array(datalist)
        vals = np.apply_along_axis(longest_run, 1, byyear)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class Drycount_seasonal(Hazard):
    def __init__(self, hazname, startdate, enddate, southern_hem, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.startdate = startdate
        self.enddate = enddate
        self.southern_hem = southern_hem
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        
        data = datalist[0] == 0
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size//365, 365)
        
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array(inseason_onerow * data.size//365)
        return (byyear * inseason) == 0

    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        vals = np.apply_along_axis(count_runs, 1, tfarray, self.min_duration)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist


class Annual_val(Hazard):
    def __init__(self, hazname, varname, aggtype, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class Seasonal_val(Hazard):
    def __init__(self, hazname, varname, aggtype, startdate, enddate, southern_hem, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.startdate = startdate
        self.enddate = enddate
        self.southern_hem = southern_hem
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array([inseason_onerow]*(data.size//365))
        byyear = byyear * inseason
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        print(vals)
        return result_dist


In [58]:
def get_observed_gee(varname, latlon, start_year, end_year, southern_hem=False):
    def relhum(T, Tdp):
        T = T.astype('float64')
        Tdp = Tdp.astype('float64')
        numerator = np.exp(17.625 * Tdp / (243.04 + Tdp))
        denominator = np.exp(17.625 * T / (243.04 + T))
        return 100 * numerator / denominator

    def get_eradata(varname, southern_hem=False):
        # Return numpy array in correct units, leapdays removed
        dataset = ee.ImageCollection("ECMWF/ERA5/DAILY")
        gee_geom = ee.Geometry.Point((latlon[1], latlon[0]))
        data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+ 1)))
        success = False
        while not success:
            try:
                d = data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()
                success = True
            except:
                print('\nRetrying')
        result = [i[4] for i in d[1:]]
        return np.array(result)
    
    if varname == 'hurs':
        success = False
        era_dewpoint = get_eradata('dewpoint_2m_temperature')-273.15
        era_maxtemp = get_eradata('maximum_2m_air_temperature')-273.15
        hist_obs = relhum(era_maxtemp, era_dewpoint)
    elif varname == 'pr':
        hist_obs = get_eradata('total_precipitation') * 1000
    elif varname == 'tasmax':
        hist_obs = get_eradata('maximum_2m_air_temperature')-273.15
    else:    # varname == 'tasmin'
        hist_obs = get_eradata('minimum_2m_air_temperature')-273.15
    return removeLeapDays(hist_obs, start_year, end_year, southern_hem)

def get_modeled_gee(varname, scenario, model, lat, lon, southern_hem, start_year, end_year):
    # Return numpy array in correct units, leapdays removed
    dataset = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.eq('model', model)).filter(ee.Filter.eq('scenario', scenario))
    gee_geom = ee.Geometry.Point((lon, lat))
    if start_year >= 2015:
        if southern_hem:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-07-01'.format(end_year)))
        else:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+ 1)))
        result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
    else:
        hist_dataset = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.eq('model', model))
        if southern_hem:
            hist_part = hist_dataset.select(varname).filter(ee.Filter.eq('scenario', 'historical')).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '2015-01-01'))
            if end_year >= 2015:
                ssp_part = dataset.select(varname).filter(ee.Filter.eq('scenario', scenario)).filter(ee.Filter.date('2015-01-01', '{0}-07-01'.format(end_year)))
        else:
            hist_part = hist_dataset.select(varname).filter(ee.Filter.eq('scenario', 'historical')).filter(ee.Filter.date('{0}-01-01'.format(start_year), '2015-01-01'))
            if end_year >= 2015:
                ssp_part = dataset.select(varname).filter(ee.Filter.eq('scenario', scenario)).filter(ee.Filter.date('2015-01-01'.format(start_year-1), '{0}-01-01'.format(end_year+ 1)))
        hist_result = [i[4] for i in hist_part.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        if end_year >= 2015:
            ssp_result = [i[4] for i in ssp_part.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        else:
            ssp_result = []
        result = hist_result + ssp_result
    d =  VARIABLES[varname]['nex_transform'](np.array(result))
    return removeLeapDays(d, start_year, end_year, southern_hem)

In [56]:
def get_calibfxns(varname, latlon, start_year, end_year):
    hist_obs = get_observed_gee(varname, latlon, start_year, end_year, southern_hem=False)
    hist_mods = {}
    rmsds = []
    for model in MODELS:
        hist_mod = get_modeled_gee(varname, 'historical', model, latlon[0], latlon[1], False, start_year, end_year)
        hist_mods[model] = hist_mod
        rmsds.append((get_rmsd(hist_obs, hist_mod), model))
    rmsds.sort()
    best_models = []
    families = []
    idx = 0
    while len(best_models) < 3:
        if not MODEL_INFO[rmsds[idx][1]] in families:
            best_models.append(rmsds[idx][1])
            families.append(MODEL_INFO[rmsds[idx][1]])
        idx += 1

# Get calibration functions
    calib_fxns = {}
    for model in best_models:
        o_quarters = quarters(hist_obs, HIST_START, HIST_END)
        m_quarters = quarters(hist_mods[model], HIST_START, HIST_END)
        calib_fxns[model] = [calibration_function(o_quarters[i].flatten(), m_quarters[i].flatten()) for i in range(4)]
    return calib_fxns


In [89]:
def getdata(varnames, scenario, lat, lon, start_year, end_year, best_models):
    return {varname: {model: get_modeled_gee(varname, scenario, model, lat, lon, lat < 0, start_year, end_year) for model in best_models[varname]} for varname in varnames}

def do_locationhazard(hazard, datasets, latlon, scenario, start_year, end_year, calib_fxns):
    lat, lon = latlon
    varnames = hazard.varname.split('+')
    return lat, lon, hazard.hazname, scenario, '{0}-{1}'.format(start_year, end_year), hazard.get_expectedval(latlon, start_year, end_year, datasets, calib_fxns), hazard.get_exceedanceprob(latlon, start_year, end_year, datasets, calib_fxns)

In [79]:
high_90c = calendardate_percentiles('tasmax', 90, CAMPINAS_LATLON, sh_hem=True)
low_90c = calendardate_percentiles('tasmin', 90, CAMPINAS_LATLON, sh_hem=True)
hot_95y = wholeyear_percentile('tasmax', 95, CAMPINAS_LATLON)
#cold_5y = wholeyear_percentile('tasmin', 5, CAMPINAS_LATLON)
#cold_10y = wholeyear_percentile('tasmin', 10, CAMPINAS_LATLON)
#ari_95y = wholeyear_percentile('ari', 95, CAMPINAS_LATLON)

In [59]:
VARNAMES = ['tasmax', 'tasmin', 'pr']
lat, lon = CAMPINAS_LATLON
calib_fxns = {varname: get_calibfxns(varname, (lat, lon), HIST_START, HIST_END) for varname in VARNAMES}
best_models = {varname: list(calib_fxns[varname].keys()) for varname in VARNAMES}

1980 2014
1980 2014
1980 2014
1980 2014


In [98]:
%%time
lat, lon = CAMPINAS_LATLON
HAZARDS = [
    Tempwave_highlow_count('Heat wave', high_90c, low_90c, 3, 1),
    Tempwave_highlow_duration('Heat wave', high_90c, low_90c, 3, 5),
    ThresholdDays('Days warmer than 25', 'tasmax', 25, True, 10),
    ThresholdDays('Days warmer than than 95th pctle yearlong', 'tasmax', hot_95y, True, 10),
    Annual_val('Hottest annual temp', 'tasmax', 'max', 35),
    Annual_val('Highest daily precip', 'pr', 'max', 500),
]
future_years = [(2020, 2030), (2030, 2040), (2040, 2050)]
scenario = 'ssp585'
results = []
for hazard in HAZARDS:
    for years in future_years:
        start_year, end_year = years
        varnames = hazard.varname.split('+')
        datasets = getdata(varnames, scenario, lat, lon, start_year, end_year, best_models)
        results.append(do_locationhazard(hazard, datasets, (lat, lon), scenario, start_year, end_year, calib_fxns))


CPU times: total: 3.53 s
Wall time: 21.9 s


In [83]:
oname = 'campinas_newnewnew.csv'
with open(oname, 'w') as ofile:
    ofile.write('latitude,longitude,hazardname,scenario,model,years,lowEV,meanEV,highEV,lowprob,meanprob,highprob\n')
for res in results:
    lat, lon, hazardname, scenario, years, evdict, probdict = res
    for modelplus in evdict.keys():
        with open(oname, 'a') as ofile:
            evvals = evdict[modelplus]
            probvals = probdict[modelplus]
            ofile.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11}\n'.format(lat, lon, hazardname, scenario, modelplus, years, evvals[0], evvals[1], evvals[2], probvals[0], probvals[1], probvals[2]))